In [1]:
using Distributed
N = 5
addprocs(N*3)

15-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16

In [2]:
nprocs()

16

In [3]:
using PyCall
cma = pyimport("cma")

PyObject <module 'cma' from '/home/mkoehler/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [5]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst
using CSV

In [37]:
@everywhere function eval_model(input_args::Array,w::Float64, input_exp::Array, output_exp::Array) 
    try 
        return Catalyst.solve(input_args[1], input_args[2], 1., 
                    input_exp, output_exp, progress=false, 
                    microcomp_type=:nonlinear, 
                    Q=input_args[3], kₙ=input_args[4], 
                    calibration=true, w=w)
    catch err #in case we loose newton convergence in a microscopic computation 
        return 1e10
    end
end

function calibrate(cma,N)
    w = [1.5302e-4*(1/0.37), 1.9128e-4*(1/0.37), 2.2954e-4*(1/0.37)] # Q8, Q10, Q12
    Init = [0.006536800306, 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0], [1, 50, 20, 20]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    #Step 1
    #Read in all 3 different velocity curves
    input_exp_12 = []
    output_exp_12 = []
    input_exp_10 = []
    output_exp_10 = []
    input_exp_8 = []
    output_exp_8 = []
    
    for row in CSV.File(datadir("experiment/SBA15_interp_12.csv"); delim = ",")
        push!(input_exp_12, row.I)
        push!(output_exp_12, row.O)
    end
    
    for row in CSV.File(datadir("experiment/SBA15_interp_10.csv"); delim = ",")
        push!(input_exp_10, row.I)
        push!(output_exp_10, row.O)
    end
    
    for row in CSV.File(datadir("experiment/SBA15_interp_8.csv"); delim = ",")
        push!(input_exp_8, row.I)
        push!(output_exp_8, row.O)
    end
    
#    microMesh = Parser.getGrid(projectdir("test/catalyst.msh"))

    #Step 2 change the model evaluation
    while isempty(es.stop()) 
        solutions = es.ask() # Returns array of array where inner array holds set of parameters length outer array = N
        #Check the syntax here
        F = Array{Future}(undef, N*3)
        errors = zeros(N*3)
        
        #Complete the for loop
        s = 1
        for idx in 1:length(errors)
            if idx % 3 == 1
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[3], input_exp_12, output_exp_12))
            elseif idx % 3 == 2
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[2], input_exp_10, output_exp_10))
            elseif idx % 3 == 0
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[1], input_exp_8, output_exp_8))
                s += 1
            end
        end
       
        
        # fetch the errors entry 
        for i in 1:N*3
            errors[i] = fetch(F[i])
        end
                
        # Reduce the N*3 length array errors to an array of length N (adding up errors)
        errors_red = sum.(Iterators.partition(errors,3))
                 
        es.tell(solutions,errors_red) 
        #This is only for saving/logging/printing current status
        es.logger.add() 
        es.disp() 
    end
return errors_red
end

calibrate (generic function with 1 method)

In [38]:
@time calibrate(cma, 2)

UndefRefError: UndefRefError: access to undefined reference

In [26]:
using Distributed
N = 6
##addprocs(N)
errors = zeros(N)
F = Array{Future}(undef,N)
for idx in 1:length(errors)        
    F[idx] = @spawnat idx+1 (println("from processor ", myid()))                   
end

for i in 1:N
    errors[i] = fetch(F[i])
end
errors

      From worker 2:	from processor 2
      From worker 4:	from processor 4
      From worker 6:	from processor 6
      From worker 5:	from processor 5
      From worker 3:	from processor 3


MethodError: MethodError: Cannot `convert` an object of type Nothing to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

      From worker 7:	from processor 7


In [36]:
@everywhere function eval_model(input_args::Array,w::Float64, input_exp::Array, output_exp::Array) 
    try 
        return Catalyst.solve(input_args[1], input_args[2], 1., 
                    input_exp, output_exp, progress=false, 
                    microcomp_type=:nonlinear, 
                    Q=input_args[3], kₙ=input_args[4], 
                    calibration=true, w=w)
    catch err #in case we loose newton convergence in a microscopic computation 
        return 1e10
    end
end

function calibrate(cma,N)
    w = [1.5302e-4*(1/0.37), 1.9128e-4*(1/0.37), 2.2954e-4*(1/0.37)] # Q8, Q10, Q12
    Init = [0.006536800306, 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0], [1, 50, 20, 20]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    #Step 1
    #Read in all 3 different velocity curves
    input_exp_12 = []
    output_exp_12 = []
    input_exp_10 = []
    output_exp_10 = []
    input_exp_8 = []
    output_exp_8 = []
    
        for row in CSV.File(datadir("experiment/SBA15_interp_12.csv"); delim = ",")
            push!(input_exp_12, row.I)
            push!(output_exp_12, row.O)
        end
    
        for row in CSV.File(datadir("experiment/SBA15_interp_10.csv"); delim = ",")
            push!(input_exp_10, row.I)
            push!(output_exp_10, row.O)
        end
    
        for row in CSV.File(datadir("experiment/SBA15_interp_8.csv"); delim = ",")
            push!(input_exp_8, row.I)
            push!(output_exp_8, row.O)
        end
    
#    microMesh = Parser.getGrid(projectdir("test/catalyst.msh"))

    #Step 2 change the model evaluation
    while isempty(es.stop()) 
        solutions = es.ask() # Returns array of array where inner array holds set of parameters length outer array = N
        #Check the syntax here
        F = Array{Future}(undef, N*3)
        errors = zeros(N*3)
        
        #Complete the for loop
                   t2 = @spawnat 2 (eval_model(solutions[1], w[3], input_exp_12, output_exp_12))
        
                   t3 = @spawnat 3 (eval_model(solutions[1], w[2], input_exp_10, output_exp_10))
                
                   t4 = @spawnat 4 (eval_model(solutions[1], w[1], input_exp_8, output_exp_8))
        
                   t5 = @spawnat 5 (eval_model(solutions[2], w[3], input_exp_12, output_exp_12))
        
                   t6 = @spawnat 6 (eval_model(solutions[2], w[2], input_exp_10, output_exp_10))
                
                   t7 = @spawnat 7 (eval_model(solutions[2], w[1], input_exp_8, output_exp_8))
                
      
        
        # fetch the errors entry 
        errors[1] = fetch(t2)
        errors[2] = fetch(t3)
        errors[3] = fetch(t4)
        errors[4] = fetch(t5)
        errors[5] = fetch(t6)
        errors[6] = fetch(t7)
        # Reduce the N*3 length array errors to an array of length N (adding up errors)
        errors_red = sum.(Iterators.partition(errors,3))
                 
        es.tell(solutions,errors_red) 
        #This is only for saving/logging/printing current status
        es.logger.add() 
        es.disp() 
    end
return errors_red
end
@time calibrate(cma,2)

(1,2mirr1)-aCMA-ES (mu_w=1.0,w_1=100%) in dimension 4 (seed=969024, Mon Jul 27 18:35:22 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      2 3.000000000000000e+10 1.0e+00 4.62e-01  4e-01  5e-01 0:00.0


UndefVarError: UndefVarError: errors_red not defined